In [1]:
#| default_exp components/tables

In [2]:
#| export
import fasthtml.common as fh
from fasthtml.common import FT
from fastcore.all import *
from fastbite.core import *
from fastbite.components.base import *
from fastbite.components.base_styles import *
from fastbite.components.containers import *
from fastbite.components.media import *
from enum import Enum

In [3]:
#| export
class TableT(VEnum):
    """Table styling variants for Flowbite components"""
    # Table level styles
    table_default = "w-full text-sm text-left rtl:text-right"
    
    # Container level styles
    container_default = "relative overflow-x-auto"
    container_responsive = "overflow-x-auto"
    container_shadow = "shadow-md"
    
    # Row level styles
    row_default = "bg-white border-b dark:bg-gray-800 dark:border-gray-700 border-gray-200"
    row_striped = "odd:bg-white odd:dark:bg-gray-900 even:bg-gray-50 even:dark:bg-gray-800 border-b dark:border-gray-700 border-gray-200"
    row_hover = "hover:bg-gray-50 dark:hover:bg-gray-600"
    row_bordered = "border-b dark:border-gray-700 border-gray-200"
    row_header = "text-xs text-gray-700 uppercase bg-gray-50 dark:bg-gray-700 dark:text-gray-400"
    row_footer = "font-semibold text-gray-900 dark:text-white"

    # Cell level styles
    cell_default = "px-6 py-4 text-gray-500 dark:text-gray-400"
    cell_expand = "w-full"
    cell_shrink = "w-1"
    cell_first = "font-medium text-gray-900 whitespace-nowrap dark:text-white"
    cell_header = "px-6 py-3"
    cell_footer = "px-6 py-4"

def Thead(*c, 
          cls:Enum|str|tuple=TableT.row_header, 
          **kwargs):
    "Creates a table header with Flowbite styling"
    # Extract any rounding classes from the parent table and apply only top rounding
    all_cls = stringify(cls).split()
    rounding_classes = [c.replace('rounded', 'rounded-t') for c in all_cls if 'rounded' in c]
    other_classes = [c for c in all_cls if 'rounded' not in c]
    
    return fh.Thead(*c, cls=(stringify(other_classes), *rounding_classes), **kwargs)

def Table(*c, # Components (typically `Thead`, `Tbody`, `Tfoot`)
          cls:Enum|str|tuple=TableT.table_default, # Classes for the table element itself
          container_cls:Enum|str|tuple=TableT.container_default, # Classes for the outer container
          responsive=True, # Whether to make the table responsive
          with_shadow=False, # Whether to add shadow styling
          **kwargs # Additional args for the table
         )->FT:
    "Creates a table with Flowbite styling"
    
    # Build container classes
    container_classes = [container_cls]
    if responsive: container_classes.append(TableT.container_responsive)
    if with_shadow: container_classes.append(TableT.container_shadow)
    
    return Div(
        fh.Table(*c, cls=stringify(cls), **kwargs),
        cls=tuple(container_classes)
    )

def _TableCell(Component, 
               *c, # Components that go in the cell
               cls:Enum|str|tuple=(), # Additional classes on the cell
               is_header=False, # Whether this is a header cell
               is_footer=False, # Whether this is a footer cell
               expand=False, # Whether to expand the cell
               shrink=False, # Whether to shrink the cell
               **kwargs # Additional args for the cell
              )->FT:
    "Creates a table cell with Flowbite styling"
    classes = []
    base_cls = TableT.cell_header if is_header else TableT.cell_default if not is_footer else TableT.cell_footer
    classes.append(base_cls)
    
    if expand:
        classes.append(TableT.cell_expand)
    if shrink:
        classes.append(TableT.cell_shrink)
    if cls:
        classes.append(stringify(cls))
    
    return Component(*c, cls=tuple(classes), **kwargs)

@delegates(_TableCell, but=['Component'])
def Td(*c, **kwargs):
    "Creates a table data cell"
    return _TableCell(fh.Td, *c, **kwargs)

@delegates(_TableCell, but=['Component'])
def Th(*c, scope='col', **kwargs):
    "Creates a table header cell"
    return _TableCell(fh.Th, *c, scope=scope, is_header=True, **kwargs)

@delegates(_TableCell, but=['Component'])
def Tf(*c, **kwargs):
    "Creates a table footer cell"
    return _TableCell(fh.Td, *c, is_footer=True, **kwargs)

def Tbody(*c,
          cls:Enum|str|tuple=(), 
          striped=False,
          hover=False,
          **kwargs):
    "Creates a table body with Flowbite styling"
    classes = []
    if striped: classes.append(TableT.row_striped)
    if hover: classes.append(TableT.row_hover)
    if cls: classes.append(stringify(cls))
    return fh.Tbody(*c, cls=tuple(classes), **kwargs)

def Tr(*c,
       cls:Enum|str|tuple=TableT.row_default,
       bordered=True,
       header=False,
       **kwargs):
    "Creates a table row with Flowbite styling"
    if header: 
        if cls == TableT.row_default: cls = TableT.row_header
        else: cls = (cls, TableT.row_header)

    if bordered: cls = (cls, TableT.row_bordered)
    return fh.Tr(*c, cls=stringify(cls), **kwargs)

def TrHeader(*c,
       cls:Enum|str|tuple=TableT.cell_header,
       bordered=True,
       **kwargs):
    "Creates a table header row with Flowbite styling"
    if bordered: cls = (cls, TableT.row_bordered)
    return fh.Tr(*c, cls=stringify(cls), **kwargs)

def Tfoot(*c,
          cls:Enum|str|tuple=TableT.row_footer,
          **kwargs):
    "Creates a table footer with Flowbite styling"
    return fh.Tfoot(*c, cls=stringify(cls), **kwargs)

def DataTable(headers:list, # List of header labels
             rows:list, # List of row data
             footer:list=None, # List of footer data
             id:str=None, # Optional ID for the table
             first_col_header=False, # Whether to style first column as header
             expand_column=None, # Index of column to expand (0-based)
             cls:Enum|str|tuple=TableT.table_default, # Classes for the table element
             container_cls:Enum|str|tuple="", # Classes for the outer container
             row_cls:Enum|str|tuple="", # Classes to apply to each row
             striped=False, # Whether to apply striped styling to rows
             hover=False, # Whether to apply hover styling to rows
             bordered=True, # Whether to apply borders between rows
             searchable=False, # Whether to apply searchable styling to rows
             sortable=False, # Whether to apply sortable styling to rows
             th_cls:Enum|str|tuple=TableT.cell_first, # Added th_cls for first col header
             tbody_cls:Enum|str|tuple="", # Added tbody_cls
             **kwargs # Additional args for the Table
            )->FT:
    "Creates a data table from headers and rows"
    # Create header row
    header_cells = []
    for i, h in enumerate(headers):
        header_cells.append(Th(h, expand=(i == expand_column)))
    header_row = TrHeader(*header_cells)
    # Create footer row
    if footer:
        footer_cells = []
        for i, f in enumerate(footer):
            footer_cells.append(Tf(f, expand=(i == expand_column)))
        footer_row = Tfoot(*footer_cells)
    else:
        footer_row = None
    # Create data rows
    data_rows = []
    for row in rows:
        cells = []
        for i, cell in enumerate(row):
            if first_col_header and i == 0:
                cells.append(Th(cell, scope='row', 
                              cls=stringify(th_cls), # Use stringify
                              expand=(i == expand_column)))
            else:
                cells.append(Td(cell, expand=(i == expand_column)))
        
        # Build row classes properly
        row_classes = stringify(TableT.row_default)  # Start with default
        if row_cls: 
            row_classes += " " + stringify(row_cls)
        if bordered: 
            row_classes += " " + stringify(TableT.row_bordered)
        if striped:
            row_classes += " " + stringify(TableT.row_striped)
        
        data_rows.append(Tr(*cells, cls=row_classes)) # Use combined row_classes
    
    # Build tbody classes properly
    tbody_classes = stringify(tbody_cls) # Start with user provided
    if striped: tbody_classes += " " + stringify(TableT.row_striped)
    if hover: tbody_classes += " " + stringify(TableT.row_hover)
    
    return Table(
        Thead(header_row),
        Tbody(*data_rows, cls=tbody_classes.strip() if tbody_classes else None), # Use combined tbody_classes
        Tfoot(footer_row) if footer else None,
        fh.Script(f"""
            if (document.getElementById('{id}') && typeof simpleDatatables.DataTable !== 'undefined') {{
                const dataTable = new simpleDatatables.DataTable('#{id}', {{
                    searchable: {str(searchable).lower()},
                    sortable: {str(sortable).lower()}
                }});
            }}
            """) if id else None,
        id=id,
        cls=stringify(cls),
        container_cls=stringify(container_cls),
        **kwargs
    )
    
# Example usage of the table components:
def SimpleTable(headers:list, rows:list, **kwargs):
    "Creates a simple table with hover effect and striped rows"
    return DataTable(
        headers=headers,
        rows=rows,
        with_shadow=True,
        first_col_header=True,
        **kwargs
    )

In [4]:
#| hide
import nbdev; nbdev.nbdev_export()